In [1]:
import pandas as pd
import os
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint

In [2]:
games_df = pd.concat(map
                     (pd.read_csv, 
                          ['data/game_details/nba_game_detailsSeason=2021-22.csv', 
                           'data/game_details/nba_game_detailsSeason=2020-21.csv', 
                           'data/game_details/nba_game_detailsSeason=2019-20.csv',
                           'data/game_details/nba_game_detailsSeason=2018-19.csv',
                           'data/game_details/nba_game_detailsSeason=2017-18.csv',
                           'data/game_details/nba_game_detailsSeason=2016-17.csv',
                           'data/game_details/nba_game_detailsSeason=2015-16.csv',
                           'data/game_details/nba_game_detailsSeason=2014-15.csv',
                           'data/game_details/nba_game_detailsSeason=2013-14.csv',
                           'data/game_details/nba_game_detailsSeason=2012-13.csv']), ignore_index=True)
games_df.head(10) 

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
0,22021,1610612746,LAC,LA Clippers,22101083,2022-03-22,LAC @ DEN,L,240,42,...,29,35,27,3,7,6,19,115,-12,1
1,22021,1610612743,DEN,Denver Nuggets,22101083,2022-03-22,DEN vs. LAC,W,240,45,...,34,40,31,4,5,11,19,127,12,1
2,22021,1610612749,MIL,Milwaukee Bucks,22101082,2022-03-22,MIL vs. CHI,W,240,50,...,48,53,26,4,6,15,12,126,28,1
3,22021,1610612741,CHI,Chicago Bulls,22101082,2022-03-22,CHI @ MIL,L,240,39,...,28,33,21,9,4,7,12,98,-28,1
4,22021,1610612737,ATL,Atlanta Hawks,22101081,2022-03-22,ATL @ NYK,W,240,40,...,33,43,26,7,5,14,18,117,6,1
5,22021,1610612752,NYK,New York Knicks,22101081,2022-03-22,NYK vs. ATL,L,240,42,...,27,41,20,12,4,14,23,111,-6,1
6,22021,1610612744,GSW,Golden State Warriors,22101080,2022-03-22,GSW @ ORL,L,240,34,...,36,48,21,6,3,18,23,90,-4,1
7,22021,1610612753,ORL,Orlando Magic,22101080,2022-03-22,ORL vs. GSW,W,240,34,...,33,39,25,9,6,14,14,94,4,1
8,22021,1610612750,MIN,Minnesota Timberwolves,22101079,2022-03-21,MIN @ DAL,L,240,36,...,35,50,21,9,3,9,18,108,-2,1
9,22021,1610612742,DAL,Dallas Mavericks,22101079,2022-03-21,DAL vs. MIN,W,240,40,...,40,47,27,5,3,12,21,110,2,1


In [3]:
games_df.head(50)

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
0,22021,1610612746,LAC,LA Clippers,22101083,2022-03-22,LAC @ DEN,L,240,42,...,29,35,27,3,7,6,19,115,-12,1
1,22021,1610612743,DEN,Denver Nuggets,22101083,2022-03-22,DEN vs. LAC,W,240,45,...,34,40,31,4,5,11,19,127,12,1
2,22021,1610612749,MIL,Milwaukee Bucks,22101082,2022-03-22,MIL vs. CHI,W,240,50,...,48,53,26,4,6,15,12,126,28,1
3,22021,1610612741,CHI,Chicago Bulls,22101082,2022-03-22,CHI @ MIL,L,240,39,...,28,33,21,9,4,7,12,98,-28,1
4,22021,1610612737,ATL,Atlanta Hawks,22101081,2022-03-22,ATL @ NYK,W,240,40,...,33,43,26,7,5,14,18,117,6,1
5,22021,1610612752,NYK,New York Knicks,22101081,2022-03-22,NYK vs. ATL,L,240,42,...,27,41,20,12,4,14,23,111,-6,1
6,22021,1610612744,GSW,Golden State Warriors,22101080,2022-03-22,GSW @ ORL,L,240,34,...,36,48,21,6,3,18,23,90,-4,1
7,22021,1610612753,ORL,Orlando Magic,22101080,2022-03-22,ORL vs. GSW,W,240,34,...,33,39,25,9,6,14,14,94,4,1
8,22021,1610612750,MIN,Minnesota Timberwolves,22101079,2022-03-21,MIN @ DAL,L,240,36,...,35,50,21,9,3,9,18,108,-2,1
9,22021,1610612742,DAL,Dallas Mavericks,22101079,2022-03-21,DAL vs. MIN,W,240,40,...,40,47,27,5,3,12,21,110,2,1


In [6]:
# Creating rows for dataframe
trans_df = pd.DataFrame()
y = []
for i in range(0,5000):#len(games_df)):
    # Check Matchup to see which team is Home
    if "@" in games_df["MATCHUP"][i]:
        away_team = games_df["TEAM_ABBREVIATION"][i]
        home_team = games_df["MATCHUP"][i][-3:]
        
        #Adding win/loss to home team
        if games_df["PLUS_MINUS"][i] > 0:
            y.append(0)
        else:
            y.append(1)
         
    else:
        home_team = games_df["TEAM_ABBREVIATION"][i]
        away_team = games_df["MATCHUP"][i][-3:]
        
        #Adding win/loss to home team
        if games_df["PLUS_MINUS"][i] > 0:
            y.append(1)
        else:
            y.append(0)
    # row list for df
    row = []
    
    #Last 10 games df for both teams
    last_10h = games_df[games_df["TEAM_ABBREVIATION"] == home_team][games_df["GAME_DATE"]< games_df["GAME_DATE"][i]].head(10)
    last_10a = games_df[games_df["TEAM_ABBREVIATION"] == away_team][games_df["GAME_DATE"]< games_df["GAME_DATE"][i]].head(10)

    needed_features = ["FGA", "FG_PCT", "FG3_PCT", "DREB", "REB", "AST"]
    
    for feature in needed_features:
        row.append(round((sum(last_10h[feature])/10),2))
        row.append(round((sum(last_10a[feature])/10),2))
    
    row.append(round((len(last_10h[last_10h["WL"]=="W"])/10),2))
    row.append(round((len(last_10a[last_10a["WL"]=="W"])/10),2))
    
    trans_df = trans_df.append(pd.DataFrame([row]), ignore_index=True)

trans_df=trans_df.set_axis(["FGA_HOME","FGA_AWAY", "FG_PCT_HOME", "FG_PCT_AWAY", "FG3_PCT_HOME", "FG3_PCT_AWAY", 
                               "DREB_HOME", "DREB_AWAY", "REB_HOME", "REB_AWAY", "AST_HOME", "AST_AWAY", "WIN_RATE_HOME", "WIN_RATE_AWAY"], axis=1)
        
trans_df["WIN"] = y

trans_df.head()

C:\Users\donjo\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:29: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\donjo\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:30: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


,FGA_HOME,FGA_AWAY,FG_PCT_HOME,FG_PCT_AWAY,FG3_PCT_HOME,FG3_PCT_AWAY,DREB_HOME,DREB_AWAY,REB_HOME,REB_AWAY,AST_HOME,AST_AWAY,WIN_RATE_HOME,WIN_RATE_AWAY,WIN
0,85.2,88.9,0.49,0.44,0.34,0.39,34.5,33.6,44.3,44.1,29.1,24.3,0.6,0.4,1
1,85.2,88.9,0.49,0.44,0.34,0.39,34.5,33.6,44.3,44.1,29.1,24.3,0.6,0.4,1
2,91.5,87.1,0.48,0.48,0.41,0.34,37.7,31.3,49.4,39.3,25.3,23.7,0.8,0.3,1
3,91.5,87.1,0.48,0.48,0.41,0.34,37.7,31.3,49.4,39.3,25.3,23.7,0.8,0.3,1
4,86.8,90.8,0.44,0.47,0.38,0.35,36.2,31.1,48.6,41.9,22.0,25.9,0.5,0.6,0


In [7]:
#Creating input and output data
trans_df["WIN"] = y
y = trans_df["WIN"]
X_df = trans_df.drop(["WIN"], axis=1)

In [8]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X_df, y, random_state=48)

In [9]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [15]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
number_hidden_nodes1 = 42
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=number_hidden_nodes1, activation="relu", input_dim=number_input_features))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation = "sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 42)                630       
                                                                 
 dense_3 (Dense)             (None, 1)                 43        
                                                                 
Total params: 673
Trainable params: 673
Non-trainable params: 0
_________________________________________________________________


In [16]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [17]:
# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

'''# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch')
'''
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
118/118 [==============================] - 1s 1ms/step - loss: 0.6819 - accuracy: 0.5792
Epoch 2/100
118/118 [==============================] - 0s 1ms/step - loss: 0.6615 - accuracy: 0.5971
Epoch 3/100
118/118 [==============================] - 0s 1ms/step - loss: 0.6568 - accuracy: 0.6061
Epoch 4/100
118/118 [==============================] - 0s 1ms/step - loss: 0.6540 - accuracy: 0.6096
Epoch 5/100
118/118 [==============================] - 0s 1ms/step - loss: 0.6515 - accuracy: 0.6107
Epoch 6/100
118/118 [==============================] - 0s 1ms/step - loss: 0.6495 - accuracy: 0.6152
Epoch 7/100
118/118 [==============================] - 0s 1ms/step - loss: 0.6481 - accuracy: 0.6163
Epoch 8/100
118/118 [==============================] - 0s 1ms/step - loss: 0.6461 - accuracy: 0.6181
Epoch 9/100
118/118 [==============================] - 0s 1ms/step - loss: 0.6443 - accuracy: 0.6176
Epoch 10/100
118/118 [==============================] - 0s 1ms/step - loss: 0.6425 - accura

118/118 [==============================] - 0s 1ms/step - loss: 0.5752 - accuracy: 0.7013
Epoch 82/100
118/118 [==============================] - 0s 1ms/step - loss: 0.5731 - accuracy: 0.7040
Epoch 83/100
118/118 [==============================] - 0s 1ms/step - loss: 0.5733 - accuracy: 0.7080
Epoch 84/100
118/118 [==============================] - 0s 1ms/step - loss: 0.5720 - accuracy: 0.7080
Epoch 85/100
118/118 [==============================] - 0s 1ms/step - loss: 0.5708 - accuracy: 0.7131
Epoch 86/100
118/118 [==============================] - 0s 1ms/step - loss: 0.5707 - accuracy: 0.7051
Epoch 87/100
118/118 [==============================] - 0s 1ms/step - loss: 0.5689 - accuracy: 0.7072
Epoch 88/100
118/118 [==============================] - 0s 1ms/step - loss: 0.5692 - accuracy: 0.7072
Epoch 89/100
118/118 [==============================] - 0s 1ms/step - loss: 0.5688 - accuracy: 0.7080
Epoch 90/100
118/118 [==============================] - 0s 1ms/step - loss: 0.5678 - accuracy: 

In [19]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

40/40 - 0s - loss: 7.8467 - accuracy: 0.5552 - 46ms/epoch - 1ms/step
Loss: 7.846717834472656, Accuracy: 0.5551999807357788


## Transforming the data

In [222]:
trans_df.head()

,FGA_HOME,FGA_AWAY,FG_PCT_HOME,FG_PCT_AWAY,FG3_PCT_HOME,FG3_PCT_AWAY,DREB_HOME,DREB_AWAY,REB_HOME,REB_AWAY,AST_HOME,AST_AWAY,WIN
0,85.2,87.7,0.49,0.46,0.34,0.33,34.5,34.7,44.3,45.8,29.1,24.5,1
1,87.7,88.9,0.46,0.44,0.33,0.39,34.7,33.6,45.8,44.1,24.5,24.3,1
2,87.7,87.1,0.46,0.48,0.33,0.34,34.7,31.3,45.8,39.3,24.5,23.7,1
3,91.5,87.7,0.48,0.46,0.41,0.33,37.7,34.7,49.4,45.8,25.3,24.5,1
4,86.8,87.7,0.44,0.46,0.38,0.33,36.2,34.7,48.6,45.8,22.0,24.5,0


In [224]:
#create column with FGA difference between teams
trans_df["FGA_DIFF"]=trans_df["FGA_HOME"]-trans_df["FGA_AWAY"]
trans_df.drop(columns=["FGA_HOME","FGA_AWAY"], inplace=True)
trans_df.head()

,FG_PCT_HOME,FG_PCT_AWAY,FG3_PCT_HOME,FG3_PCT_AWAY,DREB_HOME,DREB_AWAY,REB_HOME,REB_AWAY,AST_HOME,AST_AWAY,WIN,FGA_DIFF
0,0.49,0.46,0.34,0.33,34.5,34.7,44.3,45.8,29.1,24.5,1,-2.5
1,0.46,0.44,0.33,0.39,34.7,33.6,45.8,44.1,24.5,24.3,1,-1.2
2,0.46,0.48,0.33,0.34,34.7,31.3,45.8,39.3,24.5,23.7,1,0.6
3,0.48,0.46,0.41,0.33,37.7,34.7,49.4,45.8,25.3,24.5,1,3.8
4,0.44,0.46,0.38,0.33,36.2,34.7,48.6,45.8,22.0,24.5,0,-0.9


In [ ]:
#Create column with FG percent ratio between teams
trans_df["FG_PCT_RATIO"]=trans_df["FG_PCT_HOME"]/trans_df["FG_PCT_AWAY"]
trans_df.drop(columns=["FG_PCT_HOME", "FG_PCT_AWAY"], inplace=True)

#Create column with 3-point FG percent ratio between teams
trans_df["FG3_PCT_RATIO"] = trans_df["FG3_PCT_HOME"]/trans_df["FG3_PCT_AWAY"]
trans_df.drop(columns=["FG3_PCT_HOME", "FG3_PCT_AWAY"], inplace = True)

#Create column with Defensive rebound difference between teams
trans_df["DREB_DIFF"]=trans_df["DREB_HOME"]-trans_df["DREB_AWAY"]
trans_df.drop(columns=["DREB_HOME", "DREB_AWAY"], inplace =True)

#Create column with rebound difference between teams
trans_df["REB_DIFF"]=trans_df["REB_HOME"]-trans_df["REB_AWAY"]
trans_df.drop(columns=["REB_HOME","REB_AWAY"], inplace=True)

#Create column with assist difference between teams
trans_df["AST_DIFF"]=trans_df["AST_HOME"]-trans_df["AST_AWAY"]
trans_df.drop(columns=["AST_HOME","AST_AWAY"], inplace=True)

win=trans_df["WIN"]
trans_df.head()


In [231]:
win=trans_df["WIN"]
trans_df.drop(columns=["WIN"], inplace=True)
trans_df["WIN"]=win
trans_df.head()

,FGA_DIFF,FG_PCT_RATIO,FG3_PCT_RATIO,DREB_DIFF,REB_DIFF,AST_DIFF,WIN
0,-2.5,1.065217,1.030303,-0.2,-1.5,4.6,1
1,-1.2,1.045455,0.846154,1.1,1.7,0.2,1
2,0.6,0.958333,0.970588,3.4,6.5,0.8,1
3,3.8,1.043478,1.242424,3.0,3.6,0.8,1
4,-0.9,0.956522,1.151515,1.5,2.8,-2.5,0


In [232]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X_df, y, random_state=48)

In [233]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [235]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
number_hidden_nodes1 = 50
new_nn = tf.keras.models.Sequential()

# First hidden layer
new_nn.add(tf.keras.layers.Dense(units=number_hidden_nodes1, activation="relu", input_dim=number_input_features))

# Output layer
new_nn.add(tf.keras.layers.Dense(units=1, activation = "sigmoid"))

# Check the structure of the model
new_nn.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (None, 50)                650       
                                                                 
 dense_19 (Dense)            (None, 1)                 51        
                                                                 
Total params: 701
Trainable params: 701
Non-trainable params: 0
_________________________________________________________________


In [239]:
# Compile the model
new_nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [243]:
# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"# Compile the model
new_nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

'''# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch')
'''
# Train the model
fit_model = new_nn.fit(X_train_scaled,y_train,epochs=500)

Epoch 1/500
118/118 [==============================] - 1s 1ms/step - loss: 0.5727 - accuracy: 0.7000
Epoch 2/500
118/118 [==============================] - 0s 1ms/step - loss: 0.5699 - accuracy: 0.7003
Epoch 3/500
118/118 [==============================] - 0s 1ms/step - loss: 0.5699 - accuracy: 0.7016
Epoch 4/500
118/118 [==============================] - 0s 1ms/step - loss: 0.5694 - accuracy: 0.7059
Epoch 5/500
118/118 [==============================] - 0s 1ms/step - loss: 0.5693 - accuracy: 0.7027
Epoch 6/500
118/118 [==============================] - 0s 1ms/step - loss: 0.5708 - accuracy: 0.6997
Epoch 7/500
118/118 [==============================] - 0s 1ms/step - loss: 0.5706 - accuracy: 0.6987
Epoch 8/500
118/118 [==============================] - 0s 1ms/step - loss: 0.5674 - accuracy: 0.7091
Epoch 9/500
118/118 [==============================] - 0s 1ms/step - loss: 0.5693 - accuracy: 0.6992
Epoch 10/500
118/118 [==============================] - 0s 1ms/step - loss: 0.5692 - accura

118/118 [==============================] - 0s 1ms/step - loss: 0.5558 - accuracy: 0.7136
Epoch 161/500
118/118 [==============================] - 0s 1ms/step - loss: 0.5579 - accuracy: 0.7152
Epoch 162/500
118/118 [==============================] - 0s 1ms/step - loss: 0.5583 - accuracy: 0.7112
Epoch 163/500
118/118 [==============================] - 0s 1ms/step - loss: 0.5580 - accuracy: 0.7131
Epoch 164/500
118/118 [==============================] - 0s 1ms/step - loss: 0.5560 - accuracy: 0.7152
Epoch 165/500
118/118 [==============================] - 0s 1ms/step - loss: 0.5592 - accuracy: 0.7101
Epoch 166/500
118/118 [==============================] - 0s 1ms/step - loss: 0.5560 - accuracy: 0.7133
Epoch 167/500
118/118 [==============================] - 0s 1ms/step - loss: 0.5578 - accuracy: 0.7117
Epoch 168/500
118/118 [==============================] - 0s 1ms/step - loss: 0.5550 - accuracy: 0.7152
Epoch 169/500
118/118 [==============================] - 0s 1ms/step - loss: 0.5601 - a

118/118 [==============================] - 0s 1ms/step - loss: 0.5486 - accuracy: 0.7173
Epoch 319/500
118/118 [==============================] - 0s 1ms/step - loss: 0.5477 - accuracy: 0.7136
Epoch 320/500
118/118 [==============================] - 0s 1ms/step - loss: 0.5472 - accuracy: 0.7227
Epoch 321/500
118/118 [==============================] - 0s 1ms/step - loss: 0.5510 - accuracy: 0.7117
Epoch 322/500
118/118 [==============================] - 0s 1ms/step - loss: 0.5479 - accuracy: 0.7147
Epoch 323/500
118/118 [==============================] - 0s 1ms/step - loss: 0.5489 - accuracy: 0.7184
Epoch 324/500
118/118 [==============================] - 0s 1ms/step - loss: 0.5480 - accuracy: 0.7232
Epoch 325/500
118/118 [==============================] - 0s 1ms/step - loss: 0.5491 - accuracy: 0.7152
Epoch 326/500
118/118 [==============================] - 0s 1ms/step - loss: 0.5525 - accuracy: 0.7128
Epoch 327/500
118/118 [==============================] - 0s 1ms/step - loss: 0.5493 - a

118/118 [==============================] - 0s 1ms/step - loss: 0.5411 - accuracy: 0.7213
Epoch 477/500
118/118 [==============================] - 0s 1ms/step - loss: 0.5391 - accuracy: 0.7189
Epoch 478/500
118/118 [==============================] - 0s 1ms/step - loss: 0.5433 - accuracy: 0.7224
Epoch 479/500
118/118 [==============================] - 0s 1ms/step - loss: 0.5407 - accuracy: 0.7203
Epoch 480/500
118/118 [==============================] - 0s 1ms/step - loss: 0.5409 - accuracy: 0.7261
Epoch 481/500
118/118 [==============================] - 0s 1ms/step - loss: 0.5424 - accuracy: 0.7221
Epoch 482/500
118/118 [==============================] - 0s 1ms/step - loss: 0.5373 - accuracy: 0.7237
Epoch 483/500
118/118 [==============================] - 0s 1ms/step - loss: 0.5402 - accuracy: 0.7197
Epoch 484/500
118/118 [==============================] - 0s 1ms/step - loss: 0.5410 - accuracy: 0.7176
Epoch 485/500
118/118 [==============================] - 0s 1ms/step - loss: 0.5390 - a

In [12]:
# Evaluate the model using the test data
model_loss, model_accuracy = new_nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

NameError: name 'new_nn' is not defined